<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../_static/logo_stacked.png" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 1. Param</h2></div>
<br><br>

In [ ]:
import param
import panel as pn

pn.extension('tabulator')

# Param Basics for Panel

Panel and other projects in the HoloViz ecosystem all build on Param. Param provides a framework to add validation, documentation and interactivity to a project. It is similar to more modern projects such as Pydantic but focuses primarily on providing APIs that make it easy to express complex dependencies and UI interactions.

Here we won't focus on how param works but instead try to get you to understand the fundamentals that will allow you to effectively use Panel, express interactivity and reactivity in idiomatic ways and lastly how to structure your code to make it easily reusable and avoid callback hell.

## What is a Parameter?

Parameters are objects that express the semantics of a value of an attribute on an object, e.g. the `value` of `Widget`, i.e. whether it is a number or a string. Generally Parameters are broader than Python types because they deal with semantics not the specific representation of a value. A `Parameter` object also acts as a reference to the underlying value, which is incredibly useful when trying to add interactivity to a UI.

Let's look at a parameter definition, in this case for the Panel `TextInput` widget `value` parameter:

```python
class TextInput(Widget):

    ...
    
    value = param.String(default='', allow_None=True, doc="""
        Initial or entered text value updated when <enter> key is pressed.""")
```

A `Parameter` must be defined inside a class and that class must be a subclass of `Parameterized`. We can also see that it is of type `param.String`, defines a default of `''` and allows `None` value. It also has a docstring which gives us information about it.

To inspect all the parameters on a `Parameterized` class we can view the `repr` of the `.param` namespace:

In [ ]:
pn.widgets.TextInput.param

### Accessing parameter values

Let's start by working with a `TextInput` widget:

In [ ]:
text_input = pn.widgets.TextInput(value='A string!')

text_input

We can access the current value of the widget:

In [ ]:
text_input.value

But we can also access the `Parameter` instance that acts as a proxy or reference for value:

In [ ]:
text_input.param.value

Why and how to use such a reference is something we will discover a little bit later on.

### Setting parameter values

We can also set it to a new value:

In [ ]:
text_input.value = 'Hello World!'

If we want to set multiple parameter values at once we should do so via the `.param.update` method. This ensures that anything that watches for changes on the parameters is only triggered once:

In [ ]:
text_input.param.update(value='Updated!', width=85);

### Validation

The primary purpose of parameters is to perform validation, e.g. if I try to assign an invalid value to a parameter, we will get an error:

In [ ]:
text_input.value = 3.14

### Class vs. Instance Parameters

Parameters are defined on classes and when a `Parameterized` is created the parameters defined on it will be initialized. This means that the parameter default value and any validation related attributes defined on the parameter can be modified and all instances created thereafter will reflect these changes.

In [ ]:
#pn.widgets.TextInput.param.styles.default = {'border': '1px solid black', 'padding': '5px'}

pn.widgets.TextInput()

## Parameters as references

Above we mentioned that `Parameter` objects can act as proxies or references for the underlying Parameter value. This is a powerful feature when trying to declare interactivity. Let us see what we mean by that with a simple example again based on the `TextInput` widget:

In [ ]:
text_in = pn.widgets.TextInput(value='Hello world!')
text_out = pn.pane.Markdown(text_in.param.value)

pn.Column(text_in, text_out)

Note how we were able to pass the `value` parameter to the `Markdown` pane. If you attempt to type into the `TextInput` you'll notice that this is no automatically reflected by the Markdown output.

This also works when using it for other parameters, e.g. we can add a switch to toggle the visibility of some component:

In [ ]:
visible = pn.widgets.Switch(value=True)

pn.Row(visible, pn.pane.Markdown('Hello World!', visible=visible))

Notice that we passed in the widget object directly instead of the `.param.value`. This is possible because widgets are treated as a proxy of their `value` parameter just like a `Parameter` is treated as a proxy for current value.

### Indirect Mapping

Often a widget or parameter value will not map directly to the output you want, e.g. let's say we wanted to add some formatting around the text before we render it. To achieve this we can write a function that we then `bind` the parameter value to. This gives us a bound function which continues to act as a proxy for the (now transformed/formatted) value.

In [ ]:
def format_text(text):
    return f'**Hello {text}!**'

text_input = pn.widgets.TextInput(value='World')

md = pn.pane.Markdown(pn.bind(format_text, text_input))

pn.Row(text_input, md)

### Exercise

Write a small app where you can scale the `font-size` of a `Markdown` pane with another widget, e.g. an `IntSlider`. The `font-size` can be set using the `style` parameter.

<detail>
    Hint: The `style` parameter only accepts dictionaries of strings.
</detail>

## Writing Parameterized Classes

One last thing we should learn about working with Param in Panel is writing classes. This is useful for a number of reasons:

1. Organizing complex pieces of code and functionality
2. Writing reusable components made up of multiple Panel objects

A Parameterized class has to inherit from `param.Parameterized` and should declare one or more parameters.